In [145]:
# Make some trial data
num_words = 50
num_sentences = 20

import itertools
def takesome(iterable, number):
    ret = []
    index = 0
    while index < number:
        ret.append( next(iterable) )
        index += 1
    return ret

#words = takesome(itertools.product("abcdefghijklmnopqrstuvwxyz", repeat = 5), num_words)
#words = ["".join(x) for x in words]
words = [str(x) for x in range(num_words)]
import random
firstlen = min(1000, num_words//2)
sentences = [random.sample(words, firstlen), random.sample(words, firstlen)]
otherlen = min(10, num_words//3)
sentences.extend( random.sample(words, otherlen) for _ in range(num_sentences - 2) )
sentences = [" ".join(x) for x in sentences]

print("Trial data contains {} sentences and {} unique words.".format(len(sentences),
        len( set( w for sen in sentences for w in sen.split() )) ) )

Trial data contains 20 sentences and 50 unique words.


In [146]:
def solve_brute(sentences):
    """Brute force solution trying all assignments."""
    N = len(sentences) - 2
    words = list(set( w for sen in sentences for w in sen.split() ))
    # word_edges[i] = list of j such that words[i] in sentences[j]
    word_edges = [ [ i for i, sen in enumerate(sentences) if w in sen.split() ] for w in words ]
    minimal_both = -1
    for n in range(2**N):
        assignment = [0,1] + [ (n >>i ) & 1 for i in range(N) ]
        both = [wi for wi, _ in enumerate(words)
                if len(set( assignment[i] for i in word_edges[wi] )) > 1 ]
        if minimal_both == -1 or minimal_both < len(both):
            minimal_both = len(both)
    return minimal_both

solve_brute(sentences)

48

In [ ]:
def solve(sentences):
    """Solves using the graph theory idea."""
    words = list(set( w for sen in sentences for w in sen.split() ))
    sen_edges = [ list(set( words.index(w) for w in sen.split() )) for sen in sentences ]
    edges = []
    

In [137]:
# Build graph
# Input in `sentences`
words = list(set( w for sen in sentences for w in sen.split() ))
sen_edges = [ list(set( words.index(w) for w in sen.split() )) for sen in sentences ]
# Build inverse list: word_edges[i] is those sentences which contain word i
word_edges = [ [ i for i, sen in enumerate(sentences) if w in sen.split() ] for w in words ]

In [138]:
print(sentences)
print(words)
print(sen_edges)
print(word_edges)

['45 13 35 48 16 22 3 30 17 1 21 14 23 47 37 24 9 4 10 0 8 42 28 5 34', '9 11 36 26 49 4 34 5 45 1 27 38 30 28 46 7 42 8 10 13 6 23 40 16 20', '21 0 23 4 25 15 43 45 9 13', '35 3 37 42 38 46 24 43 16 1', '18 43 28 21 3 11 24 40 47 6', '3 26 5 37 46 29 28 12 13 43', '3 16 44 18 30 9 37 20 48 17', '26 19 16 22 33 8 13 28 4 47', '33 38 9 13 17 40 20 10 12 30', '35 34 37 38 4 5 16 13 36 28', '17 24 18 46 32 5 26 1 22 37', '14 0 31 46 16 40 10 17 20 4', '3 16 18 21 46 34 42 10 4 1', '29 38 10 5 36 12 48 41 13 17', '44 8 27 14 3 4 37 17 35 47', '49 17 45 41 19 21 26 40 4 28', '38 47 5 23 32 22 30 15 20 35', '10 7 32 26 3 6 35 37 12 39', '38 16 4 24 12 10 47 22 29 26', '5 7 20 12 28 21 48 30 1 41']
['9', '27', '40', '41', '17', '23', '7', '30', '46', '11', '24', '37', '26', '5', '18', '29', '3', '32', '22', '45', '14', '20', '35', '4', '0', '6', '36', '49', '47', '16', '38', '39', '1', '28', '34', '19', '12', '31', '25', '43', '15', '33', '44', '48', '8', '13', '42', '10', '21']
[[0, 4, 5, 7,

In [139]:
# Very simple
# To optimise, we could keep a local copy of word_edges and trim it each time we
#   add an assignment, as if a word is in both languages, we don't care about it anymore.
#   Should also keep local copy of sen_edges and trim that as well when we remove words.
class WordGraph:
    def __init__(self):
        self.assignment = {0:0, 1:1}
        
    def next_unassigned(self):
        return 1 + max(self.assignment.keys())
        
    def done(self):
        if len(self.assignment) != num_sentences:
            return -1
        return sum( len(set( self.assignment[i] for i in we )) == 1 for we in word_edges )
        
    def options_for_sentence(self, sen_num):
        linked = [j for i in sen_edges[sen_num] for j in word_edges[i] ]
        for j in linked:
            if j not in self.assignment:
                return [0, 1]
        zerocount = len([j for j in linked if self.assignment[j] == 0])
        onecount = len([j for j in linked if self.assignment[j] == 1])
        if zerocount >= onecount:
            return [0]
        return [1]
    
    def make_split(self, sen_num, assign):
        ret = WordGraph()
        ret.assignment = dict(self.assignment)
        ret.assignment[sen_num] = assign
        return ret

In [140]:
todo = [WordGraph()]
best = -1
while len(todo) > 0:
    assignment = todo.pop()
    count = assignment.done()
    if count > -1:
        if count >= best:
            best = count
            best_assignment = dict(assignment.assignment)
            print(best, assignment.assignment)
        continue
    sn = assignment.next_unassigned()
    for x in assignment.options_for_sentence(sn):
        todo.append( assignment.make_split(sn, x) )

24 {0: 0, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1}
24 {0: 0, 1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0}


In [141]:
best, best_assignment, sentences

(24,
 {0: 0,
  1: 1,
  2: 0,
  3: 0,
  4: 0,
  5: 0,
  6: 0,
  7: 0,
  8: 0,
  9: 0,
  10: 0,
  11: 0,
  12: 0,
  13: 0,
  14: 0,
  15: 0,
  16: 0,
  17: 0,
  18: 0,
  19: 0},
 ['45 13 35 48 16 22 3 30 17 1 21 14 23 47 37 24 9 4 10 0 8 42 28 5 34',
  '9 11 36 26 49 4 34 5 45 1 27 38 30 28 46 7 42 8 10 13 6 23 40 16 20',
  '21 0 23 4 25 15 43 45 9 13',
  '35 3 37 42 38 46 24 43 16 1',
  '18 43 28 21 3 11 24 40 47 6',
  '3 26 5 37 46 29 28 12 13 43',
  '3 16 44 18 30 9 37 20 48 17',
  '26 19 16 22 33 8 13 28 4 47',
  '33 38 9 13 17 40 20 10 12 30',
  '35 34 37 38 4 5 16 13 36 28',
  '17 24 18 46 32 5 26 1 22 37',
  '14 0 31 46 16 40 10 17 20 4',
  '3 16 18 21 46 34 42 10 4 1',
  '29 38 10 5 36 12 48 41 13 17',
  '44 8 27 14 3 4 37 17 35 47',
  '49 17 45 41 19 21 26 40 4 28',
  '38 47 5 23 32 22 30 15 20 35',
  '10 7 32 26 3 6 35 37 12 39',
  '38 16 4 24 12 10 47 22 29 26',
  '5 7 20 12 28 21 48 30 1 41'])

In [142]:
zero_words = list(set( int(x) for i, sen in enumerate(sentences) for x in sen.split() if best_assignment[i]==0 ))
zero_words.sort()
one_words = list(set( int(x) for i, sen in enumerate(sentences) for x in sen.split() if best_assignment[i]==1 ))
one_words.sort()

In [143]:
zero_words, one_words

([0,
  1,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49],
 [1,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  13,
  16,
  20,
  23,
  26,
  27,
  28,
  30,
  34,
  36,
  38,
  40,
  42,
  45,
  46,
  49])

In [144]:
set(zero_words).symmetric_difference(set(one_words))

{0,
 3,
 12,
 14,
 15,
 17,
 18,
 19,
 21,
 22,
 24,
 25,
 29,
 31,
 32,
 33,
 35,
 37,
 39,
 41,
 43,
 44,
 47,
 48}